In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
ace = pd.DataFrame()
for name in os.listdir('proc_data/total')[:-1]:
    ins_data = pd.read_csv(f'proc_data/total/{name}', index_col=0, parse_dates=['date'])
    ace = pd.concat([ace, ins_data], axis=0)

problem = pd.read_csv("proc_data/problem.csv", index_col=0, parse_dates=['date'])
problem.head()

,Np,Tp,Vp,B_gsm_x,B_gsm_y,B_gsm_z,Bmag
date,,,,,,,
01-01 00:00,-2484.213799,31225.352663,72.326686,5.142349,1.988692,-12.914000,14.143320
01-01 03:00,-2779.787349,53214.458580,61.408107,3.574822,-2.570586,-9.271053,11.855373
01-01 06:00,-3511.071315,42789.192262,-1991.056964,4.361542,-5.262113,-7.125196,10.517149
01-01 09:00,-9999.900000,38898.049704,-6612.537456,3.533574,-6.503805,4.220485,9.589148
01-01 12:00,-9999.900000,65806.125444,-510.487337,6.511308,-6.137467,-0.664426,9.313183


In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

print(tf.__version__, (tf.test.gpu_device_name() if tf.test.is_gpu_available() else None))

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

2.0.0-rc0 /device:GPU:0


In [4]:
ace_x, ace_y = ace.iloc[:, :-1].values, \
    keras.utils.to_categorical(ace.target.values, 10)
problem_x = tf.constant(problem.values[:, :, np.newaxis])

x_train, x_test, y_train, y_test = \
    train_test_split(ace_x, ace_y, test_size=0.2, random_state=1, shuffle=False)
x_train, x_val, y_train, y_val = \
    train_test_split(x_train, y_train, test_size=0.2, random_state=1)

ace_x, ace_y, x_train, y_train, x_val, y_val, x_test, y_test = \
    tf.constant(ace_x[:, :, np.newaxis]), tf.constant(ace_y), \
    tf.constant(x_train[:, :, np.newaxis]), tf.constant(y_train), \
    tf.constant(x_val[:, :, np.newaxis]), tf.constant(y_val), \
    tf.constant(x_test[:, :, np.newaxis]), tf.constant(y_test)

print(ace_x.shape, ace_y.shape, problem_x.shape)
print(x_train.shape, x_val.shape, x_test.shape)

(40912, 7, 1) (40912, 10) (2920, 7, 1)
(26183, 7, 1) (6546, 7, 1) (8183, 7, 1)


In [140]:
# scaler = MinMaxScaler()

# ace_x, ace_y = scaler.fit_transform(ace.iloc[:, :-1].values), \
#     keras.utils.to_categorical(ace.target.values, 10)
# problem_x = tf.constant(scaler.fit_transform(problem.values)[:, :, np.newaxis])

# x_train, x_test, y_train, y_test = \
#     train_test_split(ace_x, ace_y, test_size=0.2, random_state=1, shuffle=False)
# x_train, x_val, y_train, y_val = \
#     train_test_split(x_train, y_train, test_size=0.2, random_state=1)

# ace_x, ace_y, x_train, y_train, x_val, y_val, x_test, y_test = \
#     tf.constant(ace_x[:, :, np.newaxis]), tf.constant(ace_y), \
#     tf.constant(x_train[:, :, np.newaxis]), tf.constant(y_train), \
#     tf.constant(x_val[:, :, np.newaxis]), tf.constant(y_val), \
#     tf.constant(x_test[:, :, np.newaxis]), tf.constant(y_test)

# print(ace_x.shape, ace_y.shape, problem_x.shape)
# print(x_train.shape, x_val.shape, x_test.shape)

(40912, 7, 1) (40912, 10) (2920, 7, 1)
(26183, 7, 1) (6546, 7, 1) (8183, 7, 1)


In [19]:
optim = keras.optimizers.Adam()
loss_fn = keras.losses.MeanSquaredError()

model = keras.models.Sequential()
model.add(layers.SimpleRNN(64, input_shape=(7, 1), dropout=0.1))
model.add(layers.Dense(10, activation='linear'))
model.compile(optimizer=optim, loss=loss_fn, metrics=['acc'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_5 (SimpleRNN)     (None, 64)                4224      
_________________________________________________________________
dense_8 (Dense)              (None, 10)                650       
Total params: 4,874
Trainable params: 4,874
Non-trainable params: 0
_________________________________________________________________


In [20]:
early_stop = EarlyStopping(monitor='loss', patience=2, verbose=1)

hist = model.fit(x_train, y_train, batch_size=16, epochs=50, callbacks=[early_stop], validation_data=(x_val, y_val))
hist.params

Train on 26183 samples, validate on 6546 samples
Epoch 1/50
26183/26183 [==============================] - 16s 627us/sample - loss: 0.0820 - acc: 0.3339 - val_loss: 0.0759 - val_acc: 0.3361
Epoch 2/50
26183/26183 [==============================] - 15s 591us/sample - loss: 0.0743 - acc: 0.3531 - val_loss: 0.0726 - val_acc: 0.3740
Epoch 3/50
26183/26183 [==============================] - 15s 592us/sample - loss: 0.0736 - acc: 0.3645 - val_loss: 0.0729 - val_acc: 0.3701
Epoch 4/50
26183/26183 [==============================] - 16s 599us/sample - loss: 0.0731 - acc: 0.3684 - val_loss: 0.0728 - val_acc: 0.3894
Epoch 5/50
26183/26183 [==============================] - 16s 595us/sample - loss: 0.0729 - acc: 0.3660 - val_loss: 0.0714 - val_acc: 0.3978
Epoch 6/50
26183/26183 [==============================] - 16s 597us/sample - loss: 0.0728 - acc: 0.3677 - val_loss: 0.0725 - val_acc: 0.3584
Epoch 7/50
26183/26183 [==============================] - 16s 606us/sample - loss: 0.0725 - acc: 0.3738 -

{'batch_size': 16,
 'epochs': 50,
 'steps': 1637,
 'samples': 26183,
 'verbose': 0,
 'do_validation': True,
 'metrics': ['loss', 'acc', 'val_loss', 'val_acc']}

In [22]:
pred = model.predict_classes(problem_x)
loss, acc = model.evaluate(x_test, y_test, verbose=0)

timestamp = datetime.now().strftime('%Y-%m%d_%H%M')
pred = pd.DataFrame(pred.reshape(365, 8), index=range(1, 366), columns=[f"kp_{_}h" for _ in range(0, 24, 3)])
pred.to_csv(f"models/rnn{timestamp}(loss{round(float(loss), 3)}_acc{round(float(acc), 2)}).csv")

print(f"loss: {loss} - acc: {acc}")

loss: 0.0694235591394284 - acc: 0.40596356987953186


In [25]:
model.fit(ace_x, ace_y, batch_size=32, epochs=50, verbose=2)
loss, acc = model.evaluate(x_test, y_test, verbose=0)
timestamp = datetime.now().strftime('%Y-%m%d_%H%M')
pred = model.predict_classes(problem_x)
pred = pd.DataFrame(pred.reshape(365, 8), index=range(1, 366), columns=[f"kp_{_}h" for _ in range(0, 24, 3)])
pred.to_csv(f"models/ace_rnn{timestamp}(loss{round(float(loss), 3)}_acc{round(float(acc), 2)}).csv")

Train on 40912 samples
Epoch 1/50
40912/40912 - 11s - loss: 0.0701 - acc: 0.4012
Epoch 2/50
40912/40912 - 11s - loss: 0.0701 - acc: 0.3968
Epoch 3/50
40912/40912 - 11s - loss: 0.0703 - acc: 0.3943
Epoch 4/50
40912/40912 - 11s - loss: 0.0700 - acc: 0.4002
Epoch 5/50
40912/40912 - 11s - loss: 0.0699 - acc: 0.4025
Epoch 6/50
40912/40912 - 11s - loss: 0.0701 - acc: 0.3974
Epoch 7/50
40912/40912 - 11s - loss: 0.0701 - acc: 0.3952
Epoch 8/50
40912/40912 - 11s - loss: 0.0698 - acc: 0.4028
Epoch 9/50
40912/40912 - 11s - loss: 0.0701 - acc: 0.3971
Epoch 10/50
40912/40912 - 11s - loss: 0.0700 - acc: 0.3997
Epoch 11/50
40912/40912 - 11s - loss: 0.0699 - acc: 0.4017
Epoch 12/50
40912/40912 - 11s - loss: 0.0698 - acc: 0.4024
Epoch 13/50
40912/40912 - 11s - loss: 0.0699 - acc: 0.3997
Epoch 14/50
40912/40912 - 11s - loss: 0.0698 - acc: 0.4027
Epoch 15/50
40912/40912 - 11s - loss: 0.0700 - acc: 0.3989
Epoch 16/50
40912/40912 - 11s - loss: 0.0699 - acc: 0.4002
Epoch 17/50
40912/40912 - 11s - loss: 0.06